# Script to generate weights used in area-weighted interpolation of c404 bias corrected daily data to NHGF geospatial fabric v1.1

In [3]:
import xarray as xr
import hvplot.xarray
import hvplot.pandas
import hvplot.dask

import warnings

import datetime
import dask.bag as db
import pandas as pd
import geopandas as gpd

from holoviews.element.tiles import EsriTerrain

from gdptools.helpers import calculate_weights
from gdptools.helpers import run_weights
from gdptools.helpers import finalize_netcdf

warnings.filterwarnings('ignore')

In [4]:
# projection of bias corrected data retrieved from attributes of the crs variable
c404_proj = "+proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0 +lat_2=60.0 +lat_0=40.0 +lon_0=-97.0 +x_0=0 +y_0=0 +k_0=1.0 +nadgrids=@null +wktext  +no_defs "

In [5]:
!ls

2454094-c404.out		cache
c404_bc_daily_wghts_gfv1_1.csv	environment-dev.yml
c404_bc_wghts_gfv1_1.csv	GFv1.1_simple.cpg
c404_calc_wghts.ipynb		GFv1.1_simple.dbf
c404_calc_wghts_test.ipynb	GFv1.1_simple.prj
c404_daily_bc_conus.nc		GFv1.1_simple.shp
c404_daily_bc_conus_v2.nc	GFv1.1_simple.shx
c404_daily_bc_conus_v3.nc	gridmet_wghts.csv
c404_daily_bc_conus_v4.nc	run_weights_c404bc.ipynb
c404_wght_gen_2.bat		Untitled1.ipynb
c404_wght_gen.bat		Untitled.ipynb


In [6]:
# simplified polygons for gfv1.1
gdf = gpd.read_file('GFv1.1_simple.shp')

In [7]:
gdf.crs

<Derived Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
gdf.head()

,LAYER,GM_TYPE,OBJECTID,nhru_v1_1,hru_segme1,nhm_id,hru_id_nat,Version,Shape_Leng,Shape_Area,Change,geometry
0,NHM\nhru_v1_1_simp,Unknown Area Type,1,76127,40038,76128,76128,1.0,80441.423,188118773,-0.017302,"POLYGON ((-105544.567 804074.976, -105415.315 ..."
1,NHM\nhru_v1_1_simp,Unknown Area Type,2,76147,40038,76148,76148,1.0,53413.506,44185975,0.054540,"POLYGON ((-97185.217 806355.005, -97154.773 80..."
2,NHM\nhru_v1_1_simp,Unknown Area Type,3,76170,40021,76171,76171,1.0,54988.828,73389194,0.018316,"POLYGON ((-105894.643 815045.861, -105703.017 ..."
3,NHM\nhru_v1_1_simp,Unknown Area Type,3,76170,40021,76171,76171,1.0,54988.828,73389194,0.018316,"POLYGON ((-101865.303 821054.983, -101864.906 ..."
4,NHM\nhru_v1_1_simp,Unknown Area Type,3,76170,40021,76171,76171,1.0,54988.828,73389194,0.018316,"POLYGON ((-101865.303 821054.983, -101864.966 ..."


In [8]:
# ID of polygons
poly_idx = 'nhru_v1_1'

In [9]:
ds2 = xr.open_dataset('/caldera/hytest_scratch/scratch/conus404/conus404_daily_bc.zarr', engine='zarr')

In [10]:
ds2


<xarray.Dataset>
Dimensions:  (time: 15336, y: 3840, x: 4608)
Coordinates:
  * time     (time) datetime64[ns] 1979-10-01 1979-10-02 ... 2021-09-25
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Data variables:
    RAIN     (time, y, x) float32 ...
    T2MAX    (time, y, x) float64 ...
    T2MIN    (time, y, x) float64 ...
    crs      |S1 ...
Attributes:
    NWM_version_number:         v2.1
    model_configuration:        CONUS404_tpBiasCorr
    model_initialization_time:  1979-09-30_23:00:00
    model_output_type:          forcing
    model_output_valid_time:    1979-10-01_00:00:00
    model_total_valid_times:    744.0

In [ ]:
#gdptools functions for calculating weights

In [16]:
data_crs = c404_proj
x_coord = "x"
y_coord = "y"
t_coord = "time"
sdate = "1980-01-01T00:00:00"
edate = "2020-12-31T00:00:00"
var = "RAIN"
shp_crs = 5070
shp_poly_idx = poly_idx
wght_gen_crs = 6931

wghts = calculate_weights(
    data_file=ds2,
    data_crs=data_crs,
    x_coord=x_coord,
    y_coord=y_coord,
    t_coord=t_coord,
    sdate=sdate,
    edate=edate,
    var=var,
    shp_file=gdf,
    shp_crs=shp_crs,
    shp_poly_idx=shp_poly_idx,
    wght_gen_file='c404_bc_daily_wghts_gfv1_1.csv',
    wght_gen_crs=wght_gen_crs,
)

Reprojecting to epsg:6931 finished in 84.11 second(s)
Weight generations finished in 1575.51 second(s)
